# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split


from sklearn.linear_model import LogisticRegression

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import ClusterCentroids

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)

# df['borrower_income'] = df['borrower_income'].astype('float64')
# df['num_of_accounts'] = df['num_of_accounts'].astype('float64')
# df['total_debt'] = df['total_debt'].astype('float64')

non_numerical_columns = df.columns[(df.dtypes !='float64') & (df.dtypes !='int64')]

idx_dummies = []
idx_remove = []

for idx in non_numerical_columns:
    unique_values = df[idx].unique()

    if len(unique_values) ==1:
        idx_remove.append(idx)
        print(f'\tWe will remove {idx} from feature vector')


    elif idx == 'loan_status':
        idx_remove.append(idx)
        print(f'\t We will remove {idx} from feature vector')
    
    else:
        
        print(idx)
        idx_dummies.append(idx)

homeowner
	 We will remove loan_status from feature vector


# Split the Data into Training and Testing

In [4]:
# Create our features

X = df.drop(columns=['loan_status'])
le = LabelEncoder()
le.fit(X['homeowner'])
#X['homeowner'] = le.transform(X['homeowner'])
X = pd.get_dummies(X, columns = idx_dummies)


# Create our target
y = df['loan_status'].replace({'high_risk':1, 'low_risk':0})

In [5]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [6]:
# Check the balance of our target values
Counter(y)

Counter({0: 75036, 1: 2500})

In [7]:
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = \
    train_test_split(X, y,  random_state=42)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [8]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [9]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [10]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [11]:
model = LogisticRegression(solver='lbfgs', random_state=42)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

In [12]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred, adjusted=True).round(3)

0.976

In [13]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[18690,   102],
       [   11,   581]])

In [14]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.99      0.98      1.00      0.99      0.98     18792
          1       0.85      0.98      0.99      0.91      0.99      0.97       592

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [15]:
# Resample the training data with the RandomOversampler
ros = RandomOverSampler(random_state=42)

# View the count of target classes with Counter
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

Counter({0: 56244, 1: 56244})

In [16]:
# Train the Logistic Regression model using the resampled data
model_naive_ros = LogisticRegression(solver='lbfgs', random_state=42)
model_naive_ros.fit(X_resampled, y_resampled)

y_pred = model_naive_ros.predict(X_test_scaled)

In [17]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred, adjusted=True).round(3)

0.991

In [18]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[18677,   115],
       [    2,   590]])

In [19]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.99      1.00      1.00      1.00      0.99     18792
          1       0.84      1.00      0.99      0.91      1.00      0.99       592

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384



### SMOTE Oversampling

In [20]:
# Resample the training data with SMOTE
X_resampled, y_resampled = SMOTE(random_state=42, sampling_strategy=1.0).fit_resample(
    X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({0: 56244, 1: 56244})

In [21]:
# Train the Logistic Regression model using the resampled data
model_smote = LogisticRegression(solver='lbfgs', random_state=42)
model_smote.fit(X_resampled, y_resampled)
y_pred = model_smote.predict(X_test_scaled)

In [22]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred, adjusted=True).round(3)

0.991

In [23]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[18677,   115],
       [    2,   590]])

In [24]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.99      1.00      1.00      1.00      0.99     18792
          1       0.84      1.00      0.99      0.91      1.00      0.99       592

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [25]:
# Resample the data using the ClusterCentroids resampler
cc = ClusterCentroids(random_state=42)
X_resampled, y_resampled = cc.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({0: 1908, 1: 1908})

In [26]:
# Train the Logistic Regression model using the resampled data
model_cc = LogisticRegression(solver='lbfgs', random_state=42)
model_cc.fit(X_resampled, y_resampled)
y_pred = model_cc.predict(X_test_scaled)

In [27]:
# Calculate the balanced accuracy score
balanced_accuracy_score(y_test, y_pred, adjusted=True).round(3)

0.988

In [28]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[18689,   103],
       [    4,   588]])

In [29]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.99      0.99      1.00      0.99      0.99     18792
          1       0.85      0.99      0.99      0.92      0.99      0.99       592

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [30]:
# Resample the training data with SMOTEENN
smote_enn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smote_enn.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({0: 55904, 1: 55429})

In [31]:
# Train the Logistic Regression model using the resampled data
model_smoteenn = LogisticRegression(solver='lbfgs', random_state=42)
model_smoteenn.fit(X_resampled, y_resampled)
y_pred = model_smoteenn.predict(X_test_scaled)

In [32]:
# Calculate the balanced accuracy score
balanced_accuracy_score(y_test, y_pred, adjusted=True).round(3)

0.99

In [33]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[18676,   116],
       [    2,   590]])

In [34]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.99      1.00      1.00      1.00      0.99     18792
          1       0.84      1.00      0.99      0.91      1.00      0.99       592

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384



# Final Questions

1. Which model had the best adjusted balanced accuracy score?

   * Simple Logistic Regression 0.976
   * __Naive Random Oversampling 0.991__
   * __SMOTE oversampling 0.991__
   * Undersampling 0.988
   * Combination (Over and Under) Sampling 0.99
   

2. Which model had the best recall score?

   * Simple Logistic Regression 0.98
   * __Naive Random Oversampling 1.00__
   * __SMOTE oversampling 1.00__
   * Undersampling 0.99
   * __Combination (Over and Under) Sampling 1.00__
   
3. Which model had the best geometric mean score?

   * Simple Logistic Regression 0.99
   * __Naive Random Oversampling 1.00__
   * __SMOTE oversampling 1.00__
   * Undersampling 0.99
   * __Combination (Over and Under) Sampling 1.00__
   